## word2vec
##### 単語をベクトルで表現する表現は二つある。
* カウントベース
* 推論ベース

#### カウントベース手法の問題点
* コーパス全体の統計データをしようしているので、大規模なコーパスを扱う際に学習に時間がかかってしまう。

#### 推論ベース手法
* ニューラルネットワークでミニバッチを使用する。
* 出現しうる単語を推測する。（確率分布を使用したりして）
* one-hot表現（０か１）で表す。（単語IDの該当する箇所を１、それ以外を０にする）

In [1]:
import numpy as np
import os
os.chdir("/work/NaturalProcessing/deep-learning-from-scratch-2-master")

In [2]:
from common.layers import MatMul
c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[ 1.62940445 -0.56070117 -1.2991181 ]]


### word２vecの実装
* **CBOW(continuous bag-of-words)**と呼ばれるモデルを使用する
* コンテキストからターゲットを推測することをを目的としたニューラルネットワーク
##### 流れ
* 入力層→重みの設定→順伝播→softmax with Loss(softmaxレイヤと交差エントロピーーレイヤの融合版)

In [3]:
#サンプルのコンテキストデータ（入力層が複雑になる）
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

#重みの初期化
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

#レイヤの作成（それぞれの層で重みを共有）
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

#順伝播（活性化関数を使用しない）
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)
print(s)

[[ 0.72824569 -0.87268959  0.57119892 -0.90097082  0.8411245   0.28619888
   1.51776289]]


In [4]:
from common.util import preprocess
text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)
print(id_to_word)

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [5]:
def create_contexts_target(corpus, window_size=1):
    target = corpus[window_size:-window_size]
    contexts = []
    
    for idx in range(window_size, len(corpus)-window_size):
        cs = []
        for t in range(-window_size, window_size + 1):
            if t == 0:
                continue
            cs.append(corpus[idx + t])
        contexts.append(cs)
        
    return np.array(contexts), np.array(target)

In [6]:
contexts, target = create_contexts_target(corpus, window_size=1)
print(contexts)
print("target:", target)

[[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]]
target: [1 2 3 4 1 5]


In [7]:
def convert_one_hot(corpus, vocab_size):
    N = corpus.shape[0]
    if corpus.ndim == 1:
        one_hot = np.zeros((N, vocab_size), dtype=np.int32)
        for idx, word_id in enumerate(corpus):
            one_hot[idx, word_id] = 1
    elif corpus.ndim == 2:
        C = corpus.shape[1]
        one_hot = np.zeros((N, C, vocab_size), dtype=np.int32)
        for idx_0, word_ids in enumerate(corpus):
            for idx_1, word_id in enumerate(word_ids):
                one_hot[idx_0, idx_1, word_id] = 1
    return one_hot

In [8]:
vocab_size = len(word_to_id)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)